In [ ]:
#r "nuget: VirtualPhotonics.Vts"
#r "nuget: Plotly.NET.CSharp" 
#r "nuget: Plotly.NET.Interactive"

Below, we'll reference the software packages and namespaces we'll need:

In [ ]:
using Vts.Common;
using Vts.Extensions;
using Vts.Modeling.Optimizers;
using Vts.Modeling.ForwardSolvers;
using Vts.SpectralMapping;
using Vts.Factories;
using Vts.MonteCarlo;
using Vts.MonteCarlo.Sources;
using Vts.MonteCarlo.Tissues;
using Vts.MonteCarlo.Detectors;
using Vts.MonteCarlo.Factories;
using Vts.MonteCarlo.PhotonData;
using Vts.MonteCarlo.PostProcessing;
using System.Numerics;
using Plotly.NET.CSharp;
using Plotly.NET.LayoutObjects;

Below, we'll declare a few helper functions that we'll use later for convenience of aggregating data and plotting:

In [ ]:
static double[] GetMidpoints(this Vts.Common.DoubleRange endpoints)
    => endpoints.ToArray().GetMidpoints();

// static IEnumerable<T> TakeEveryNth<T>(this IEnumerable<T> values, int n, int skip = 0)
//     => values.Where((_, i) => (i - skip) % n == 0);

static Plotly.NET.GenericChart.GenericChart ScatterChart(double[] xValues, double[] yValues, string xLabel = "", string yLabel = "", string title = "")
    => Chart.Point<double, double, string>(xValues, yValues).WithStandardStyling(xLabel, yLabel, title);

static Plotly.NET.GenericChart.GenericChart LineChart(double[] xValues, double[] yValues, string xLabel = "", string yLabel = "", string title = "")
    => Chart.Line<double, double, string>(xValues, yValues).WithStandardStyling(xLabel, yLabel, title);

static Plotly.NET.GenericChart.GenericChart WithStandardStyling(
    this Plotly.NET.GenericChart.GenericChart chart, string xLabel = "", string yLabel = "", string title = "")
{
    // uses Plotly.NET.CSharp.ChartExtensions (adding Plotly.NET to the using statements above will break this)
    return chart
        .WithTraceInfo(title, ShowLegend: !string.IsNullOrWhiteSpace(title))
        .WithXAxisStyle<double, double, string>(Title: Plotly.NET.Title.init(xLabel))
        .WithYAxisStyle<double, double, string>(Title: Plotly.NET.Title.init(yLabel))
        .WithLegendStyle(X: 0, Y: 150);
}

static Plotly.NET.GenericChart.GenericChart Heatmap(
    IEnumerable<double[]> values, 
    double[] x, 
    double[] y,
    string xLabel = "",
    string yLabel = "",
    string title = "")
{
    // attn developers: for reference, the following are the type parameters used in the call to Chart2D.Chart.Heatmap:
    // Chart2D.Chart.Heatmap<a37: (row format), a38: (fluence value type), a39: X (rho value type), a40: Y (z value type), a41: Text type>(...)
    var chart = Plotly.NET.Chart2D.Chart.Heatmap<IEnumerable<double>, double, double, double, string>(
        zData: values,
        X: x, Y: y,
        ReverseScale: false, ReverseYAxis: true,
        Transpose: true,
        Text: title,
        ColorScale: Plotly.NET.StyleParam.Colorscale.Hot
    ).WithTraceInfo(title, ShowLegend: !string.IsNullOrWhiteSpace(title))
     .WithLegendStyle(X: 0, Y: 150);

    //var chartLayout = Plotly.NET.GenericChart.getLayout(chart);
    //var yAxis = Plotly.NET.Layout.getLinearAxisById(Plotly.NET.StyleParam.SubPlotId.NewYAxis(1)).Invoke(chartLayout);
    //yAxis.SetValue("scaleanchor", Plotly.NET.StyleParam.LinearAxisId.NewX(1));
    //chart = Plotly.NET.GenericChartExtensions.WithYAxis(chart, yAxis);

    chart = Plotly.NET.GenericChartExtensions
        .WithYAxis(chart, LinearAxis.init<IConvertible, IConvertible, IConvertible, IConvertible, double, IConvertible>(
            ScaleAnchor: Plotly.NET.StyleParam.LinearAxisId.NewX(1), AxisType: Plotly.NET.StyleParam.AxisType.Linear))
        .WithXAxisStyle<double, double, string>(Title: Plotly.NET.Title.init(xLabel), MinMax: new Tuple<double, double>(x[0], x[^1]))
        .WithYAxisStyle<double, double, string>(Title: Plotly.NET.Title.init(yLabel), MinMax: new Tuple<double, double>(y[0], y[^1]));

    chart = Plotly.NET.GenericChartExtensions
        .WithColorbar(chart, title: Plotly.NET.Title.init(title));

    return chart;
} 

In [ ]:
using Vts.Common; // needed again for DoubleRange for some reason
// Example 01: run a simple Monte Carlo simulation with 1000 photons.
// Notes:
//    - default source is a point source beam normally incident at the origin 
//    - default tissue is a 100mm thick slab with air-tissue boundary and optical properties: mua: 0.01, musp: 1.0, g: 0.8, n:1.4

// create a SimulationInput object to define the simulation
var detectorRange = new DoubleRange(start: 0, stop: 40, number: 201);
var simulationInput = new SimulationInput
{
    // specify the number of photons to run
    N = 1000,

    // define a single R(rho) detector by the endpoints of rho bins
    DetectorInputs = new List<IDetectorInput> { new ROfRhoDetectorInput { Rho = detectorRange, Name = "ROfRho" } }, // name can be whatever you want
};

// create the simulation
var simulation = new MonteCarloSimulation(simulationInput);

// run the simulation
var simulationOutput = simulation.Run();

// plot the results using Plotly.NET
var detectorResults = (ROfRhoDetector)simulationOutput.ResultsDictionary["ROfRho"];
var logReflectance = detectorResults.Mean.Select(r => Math.Log(r)).ToArray();
var (detectorMidpoints, xLabel, yLabel) = (detectorRange.GetMidpoints(), "ρ [mm]", "log(R(ρ)) [mm-2]");
var chart = LineChart(detectorMidpoints, logReflectance, xLabel, yLabel, title: "log(R(ρ)) [mm-2]");

chart.Show();